In [18]:
from ultralytics import SAM, YOLO
import os
import cv2
import torch
import numpy as np


In [13]:
img = cv2.imread('assets/example.jpg')
img.shape

(599, 800, 3)

In [14]:
model = YOLO('yolo11x-seg.pt') 

In [30]:
results = model('https://ultralytics.com/images/bus.jpg')
# 3. Lưu kết quả
for result in results:
    # Lưu ảnh đã vẽ đè tất cả các mask lên
    result.save(filename='result_all.jpg')
    
    # In ra số lượng vật thể tìm được
    print(f"Số lượng vật thể phát hiện: {len(result.masks)}")
    print(result.masks.shape)

print("Đã xong. Kiểm tra file 'result_all.jpg'")


image 1/1 /workspace/VLM2Vec/bus.jpg: 640x480 4 persons, 1 bus, 69.2ms
Speed: 9.4ms preprocess, 69.2ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 480)
Số lượng vật thể phát hiện: 5
torch.Size([5, 640, 480])
Đã xong. Kiểm tra file 'result_all.jpg'


In [32]:
for result in results:
    # In ra số lượng vật thể tìm được
    print(f"Số lượng vật thể phát hiện: {len(result.masks)}")
    masks = result.masks.data.cpu().numpy()
    print(masks.shape)

print("Đã xong. Kiểm tra file 'result_all.jpg'")

Số lượng vật thể phát hiện: 5
(5, 640, 480)
Đã xong. Kiểm tra file 'result_all.jpg'
